# CHIPPR

This notebook demonstrates the use of the Cosmological Hierarchical Inference with Probabilistic Photometric Redshifts (CHIPPR) package to estimate the redshift density function.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import chippr
from chippr import *

In [ ]:
help(chippr)

## Simulation

In [ ]:
tru_amps = np.array([0.20, 0.35, 0.55])
tru_means = np.array([0.5, 0.2, 0.75])
tru_sigmas = np.array([0.4, 0.2, 0.1])

tru_nz = gmix(tru_amps, tru_means, tru_sigmas, limits=(0., 1.))

In [ ]:
N = int(1e3)

tru_zs = tru_nz.sample(N)

plt.hist(tru_zs, bins=50, normed=1)
plt.xlabel(r'true $z$')
plt.ylabel(r'probability')

In [ ]:
params = {}

In [ ]:
lf_sigma = 0.05

params['constant_sigma'] = lf_sigma

In [ ]:
bin_ends = np.array([0., 1.])
weights = np.array([1.])

int_prior = discrete(bin_ends, weights)

In [ ]:
posteriors = catalog(params)
output = posteriors.create(tru_zs, int_prior)

data = output['interim_posteriors']

In [ ]:
print(np.shape(np.sum(data, axis=0)))

In [ ]:
for datum in data[:10]:
    plt.plot(posteriors.x_coarse, datum)
plt.plot(posteriors.x_coarse, int_prior.evaluate(posteriors.x_coarse), color='k')

In [ ]:
for n, z in enumerate(data[:10]):
    plt.plot(posteriors.x_coarse, data[n], 'ko')
    plt.plot(posteriors.x_fine, posteriors.obs_lfs[n], 'k-')
    plt.show()

## Inference

In [ ]:
# data_loc = '../../../prob-z/nz/code/tests/sig2/data/logdata.csv'

# with open(data_loc,'rb') as data_file:
#     tuples = (line.split(None) for line in data_file)
#     data = [[float(pair[k]) for k in range(0,len(pair))] for pair in tuples]

In [ ]:
zs = output['bin_ends']
nz_intp = output['interim_prior']
z_posts = data

print(np.shape(zs), np.shape(nz_intp), np.shape(z_posts))

z_mids = (zs[1:]+zs[:-1])/2.
n_bins = len(z_mids)

In [ ]:
prior_sigma = 0.16
prior_var = np.eye(n_bins)
for b in range(n_bins):
    prior_var[b] = 1. * np.exp(-0.5 * (z_mids[b] - z_mids) ** 2 / prior_sigma ** 2)
l = 1.e-4
prior_var = prior_var+l*np.identity(n_bins)

prior_mean = nz_intp
prior = gauss(prior_mean, prior_var)

In [ ]:
nz = log_z_dens(output, prior, truth=tru_nz, vb=True)

In [ ]:
nz_mmle = nz.optimize(np.log(nz_intp))

In [ ]:
nz.plot()

In [ ]:
nz_stack = nz.stack()

In [ ]:
nz_mmap = nz.mmap()

In [ ]:
nz_mexp = nz.mexp()

In [ ]:
#nz_samps = nz.sample()